In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
class sghmc():
    def __init__(self,data,lnp_grad,initial,C,B=0,M=None,logpri=None):
        '''
        
        '''
        self.get_data(data)
        self.get_mass_matrix(M)
        self.get_fric_term(B,C)
        self.theta0 = initial
        self.lnp_grad = lnp_grad
        if logpri is None:
            self.logpri = self.gauss_logL
        else:
            self.logpri = logpri
        
    def gauss_logL(self,theta=None,mu=None,sigma=None):
        """
        gaussian likelihood function
        """
        k = self.ndim
        if mu is None:
            mu = np.zeros(k)
        if sigma is None:
            sigma = np.identity(k)
        return -0.5*(np.log(sigma) + (theta-mu).T @ np.linalg.inv(sigma) @ (theta-mu) + k*np.log(2*np.pi))
    
    def get_data(self, data):
        """
        get the dataset with the frame of [n_observation * n_features]
        """
        self.data = data
        self.ndim = len(data[0])
        
    def get_mass_matrix(self, mass_matrix=None):
        """
        get the inverse of the mass matrix
        """
        if mass_matrix is None:
            self.mass_matrix = np.identity(self.ndim)
            self.inverse_mass_matrix = np.identity(self.ndim)
        else:
            if len(mass_matrix) != self.ndim:
                print("Invalid mass matrix")
            elif len(mass_matrix) == 1:
                self.mass_matrix = mass_matrix
                self.inverse_mass_matrix = 1. / mass_matrix
                #self.ndim_mass = 1
            else:
                self.mass_matrix = mass_matrix
                self.inverse_mass_matrix = np.linalg.inv(mass_matrix)
            #self.ndim_mass = 2
        
    def get_fric_term(self, B_est=0, C=None):
        """
        define variance of noise model and user specified friction term
        """
        self.B = B_est
        self.C = C
        
    def define_momentum(self):
        """
        sample momentum
        """
        if self.ndim == 1:
            r = np.random.normal(0, np.sqrt(self.mass_matrix))
        else:
            r = np.random.multivariate_normal(np.zeros(self.ndim), self.mass_matrix)
        return r
    
    def velocity(self, r):
        """
        Get the velocities (gradient of kinetic) given a momentum vector
        """
        if self.ndim == 1:
            v = self.inverse_mass_matrix * r
        else:
            v = np.dot(self.inverse_mass_matrix, r)
        return v

    #def kinetic_energy(self, r):
    #    """
    #    Get the kinetic energy given momentum
    #    """
    #    if self.ndim == 1:
    #        K = self.inverse_mass_matrix * r**2
    #    else:
    #        K = np.dot(r.T, np.dot(self.inverse_mass_matrix, r))
    #    return 0.5 * K
    
    def leapfrog(self, theta, r, epsilon, size):
        """Perfrom one leapfrog step, updating the momentum and position
        vectors. 
        """
        #update momentum and position vectors
        theta += epsilon * self.velocity(r)
        r1 = - 0.5 * epsilon * self.grad_U(size, theta)
        
        if self.ndim == 1:
            r2 = - epsilon * self.C * self.velocity(r)
            r3 = np.random.normal(np.zeros(self.ndim), 2*epsilon*(self.C-self.B))
        else:
            r2 = - epsilon * self.C @ self.velocity(r)
            r3 = np.random.multivariate_normal(np.zeros(self.ndim), 2*epsilon*(self.C-self.B))
        r = r + r1 + r2 + r3
        #r = r - 0.5 * epsilon * self.grad_U(size, theta) - epsilon * self.C @ self.velocity(r) + np.random.multivariate_normal(np.zeros(self.ndim), 2*epsilon*(self.C-self.B))
        return theta, r
    
    #def lnprob(self, theta, x):#。。。
    #    return self.model.lnprob(theta, x)
        
    #def lnprob_grad(self, theta):#。。。
    #    return self.model.lnprob_grad(theta)
    
    def grad_U(self, size, theta):
        """
        get the estimate gradient based on minibatches
        
        pramas size:
            number of datapoints
        """
        df = pd.DataFrame(self.data)
        batch = df.sample(n=size)#,random_state=np.random.RandomState())
        s = 0
        for x in batch:
            s += self.lnp_grad(x, theta)
        u = -self.ndim/size * s - self.logpri_grad(theta)
        
        return u
    
    def sampling(self, iterations, epsilon, length, size):
        """
        sample theta for distribution
        
        pramas theta0:
            initial value 
        
        pramas iterations:
            number of sampling (trajectory)
        
        params epsilon:
            stepsize for the leapfrog
        
        params length:
            number of leapfrog
            
        params size:
            the size of minibatches
        """
        #setup sampling storage
        self.samples = np.zeros([iterations, self.ndim])
        
        theta0 = self.theta0
        
        # loop over trajectories
        for t in range(iterations):
            r = self.define_momentum()
            theta = theta0
            self.samples[t, :] = theta
            
            #loop over leapfrog
            theta_t, r_t = theta, r 
            for i in range(length):
                theta_m, r_m = self.leapfrog(theta_t, r_t, epsilon, size)
                theta_t,r_t = theta_m, r_m
            theta0 = theta_m 

In [ ]:
temp = sghmc(a,g_grad,5,1)

In [ ]:
temp.sampling(2000,0.001,100,100)

In [ ]:
temp.samples

In [ ]:
sum(temp.samples)/len(temp.samples)

In [ ]:
sum(temp.samples)/len(temp.samples)

In [ ]:
def gauss_logL(x, sigma, mu):
    """gaussian likelihood"""
    k = len(x)
    return -0.5*(np.log(sigma) + (x-mu).T @ sigma**(-1) @ (x-mu) + k*np.log(2*np.pi))

In [ ]:
np.random.multivariate_normal(np.zeros(10), np.identity(10))

In [ ]:
a = np.random.normal(loc=2.5,scale=1,size=10000)
a = a.reshape([-1,1])

In [ ]:
def g_grad(x,theta):
    return (x-theta)/1

In [ ]:
d = len(initial)
for i in range(d):
    r[i] = ...
M
samples: df/dic/[k,m,n]


In [111]:
class hmc():
    def __init__(self,data,lnp,lnp_grad,initialguess, M = None):
        '''
        
        '''
        self.data = data 
        self.ndim = len(initialguess)
        self.get_mass_matrix(M)
        self.theta0 = initialguess
        self.lnp = lnp
        self.lnp_grad = lnp_grad
        self.res = []
        self.n = len(data)
        print(self.ndim)

    def get_mass_matrix(self, mass_matrix=None):
        """
        get the inverse of the mass matrix
        """
        if mass_matrix is None:
            self.mass_matrix = np.identity(self.ndim)
            self.inverse_mass_matrix = np.identity(self.ndim)
        else:
            if len(mass_matrix) != self.ndim:
                print("Invalid mass matrix")
            elif len(mass_matrix) == 1:
                self.mass_matrix = mass_matrix
                self.inverse_mass_matrix = 1. / mass_matrix
                #self.ndim_mass = 1
            else:
                self.mass_matrix = mass_matrix
                self.inverse_mass_matrix = np.linalg.inv(mass_matrix)
            #self.ndim_mass = 2
        
    def define_momentum(self):
        """
        sample momentum
        """
        if self.ndim == 1:
            r = np.random.normal(0, np.sqrt(self.mass_matrix))
        else:
            r = np.random.multivariate_normal(np.zeros(self.ndim), self.mass_matrix)
        return r
    
    def velocity(self, r):
        """
        Get the velocities (gradient of kinetic) given a momentum vector
        """
        if self.ndim == 1:
            v = self.inverse_mass_matrix * r
        else:
            v = np.dot(self.inverse_mass_matrix, r)
        return v

    def kinetic_energy(self, r):
        """
        Get the kinetic energy given momentum
        """
        if self.ndim == 1:
            K = self.inverse_mass_matrix * r**2
        else:
            K = np.dot(r.T, np.dot(self.inverse_mass_matrix, r))
        return 0.5 * K
    
    def grad_U(self, theta):
        """
        get the estimate gradient based on minibatches
        
        pramas size:
            number of datapoints
        """
        s = [0] * self.ndim
        for x in self.data:
            s += self.lnp_grad(x, theta)  
        return -s / self.n
    
    def leapfrog(self, theta, r, epsilon):
        """Perfrom one leapfrog step, updating the momentum and position
        vectors. 
        """
        #update momentum and position vectors
        theta += epsilon * self.velocity(r)
        r -= epsilon * self.grad_U(theta)
        #r = r - 0.5 * epsilon * self.grad_U(size, theta) - epsilon * self.C @ self.velocity(r) + np.random.multivariate_normal(np.zeros(self.ndim), 2*epsilon*(self.C-self.B))
        return theta, r
    
    
    
    def trajectory(self, theta_t, epsilon,length):
        r_t = self.define_momentum()
        theta0, r0 = theta_t.copy(), r_t.copy()
        r0 = r_t-0.5*epsilon*self.grad_U(theta0)
        for i in range(length):
            theta_m, r_m = self.leapfrog(theta0,r0,epsilon)
            theta0, r0 = theta_m, r_m
        r0 -= 0.5*epsilon*self.grad_U(theta0)
        
        
        #M-H step
        mu = np.random.uniform(size=1)
        p = np.exp(-self.U(theta0)-self.kinetic_energy(r0) + self.U(theta_t) + self.kinetic_energy(r_t))
        if mu < min(1,p):
            return theta0
        else:
            return [0]
    
    def U(self, theta):        
        s = 0
        for x in self.data:
            s += self.lnp(x, theta)
        return -s / self.n
    
    
    def sampling(self, iterations, epsilon, length, size):
        """
        sample theta for distribution
        
        pramas theta0:
            initial value 
        
        pramas iterations:
            number of sampling (trajectory)
        
        params epsilon:
            stepsize for the leapfrog
        
        params length:
            number of leapfrog
            
        params size:
            the size of minibatches
        """
        #setup sampling storage
        thetacurr = self.theta0
        # loop over trajectories
        for t in range(iterations):
            temp = self.trajectory(thetacurr, epsilon, length)
            if len(temp) == 2 or temp[0] != 0:
                self.res.append(temp)
                thetacurr = temp

In [93]:
mean = [1, -1]
cov = [[1, 0.9], [0.9, 1]]
data2 = np.random.multivariate_normal(mean, cov, 200)

In [94]:
def mugrad(x, theta): # x is 1x2, theta is also 1x2
    xn = np.array(x)
    t = np.array(theta)
    return np.linalg.inv(np.array(cov)) @ (xn - t)

In [95]:
from scipy.stats import multivariate_normal
def logprob(x, theta):
    xn = np.array(x)
    t = np.array(theta)
    return multivariate_normal.logpdf(xn,t, np.array(cov))

In [ ]:
cov

In [ ]:
def gauss_logL(x, sigma, mu):
    """gaussian likelihood"""
    k = len(x)
    return -0.5*(np.log(sigma) + (x-mu).T @ sigma**(-1) @ (x-mu) + k*np.log(2*np.pi))

In [96]:
two = hmc(data2, logprob, mugrad, [2,-3])

In [97]:
two.sampling(1000, 0.1, 5, 200)

In [98]:
res = np.zeros((len(two.res),2))
for i in range(len(two.res)):
    res[i,:] = two.res[i]

In [101]:
np.mean(res, axis = 0)

array([ 1.13160625, -0.93874911])

In [ ]:
#Example for one dimensional gaussian 

In [102]:
data = np.random.normal(loc=2.5,scale=2,size=200)
data  = data.reshape([-1,1])

In [118]:
def mugrad(x,theta):
    return (x-theta) 

In [123]:
def sigma2grad(x, theta):
    x = np.array(x)
    theta = np.array(theta)
    return ((x - 2.5) ** 2) / (2 * theta * theta) - (1 / (2 * theta))

In [114]:
def logmu(x, mu):
    return -0.5 * np.log(2*np.pi) - (((x-mu) ** 2) / 2)

In [122]:
def logsig(x, sigma2):
    xt = np.array(x)
    sig = np.array(sigma2)
    return norm.logpdf(xt, loc = 2.5, scale = sig)

In [ ]:
logsig(2.5,[1])

In [62]:
from scipy.stats import norm
norm.logpdf(2.5, loc=2.5, scale=1)

-0.9189385332046727

In [119]:
muest = hmc(data,logmu,mugrad,[10])

1


In [120]:
muest.sampling(200,0.1,5,500)

In [121]:
np.mean(muest.res)

2.260146571770293

In [124]:
sigmaest = hmc(data,logsig,sigma2grad,[5])

1


In [125]:
sigmaest.sampling(1000,0.05,10,500)

In [127]:
sigmaest.res

[array([[4.93362696]]),
 array([[4.48404622]]),
 array([[4.89495515]]),
 array([[5.295245]]),
 array([[5.93047203]]),
 array([[4.90331445]]),
 array([[5.4233789]]),
 array([[6.02199552]]),
 array([[6.6343482]]),
 array([[6.47971711]]),
 array([[7.22093634]]),
 array([[7.33219803]]),
 array([[7.36863109]]),
 array([[8.27460086]]),
 array([[7.58806611]]),
 array([[7.72041538]]),
 array([[8.6398722]]),
 array([[8.46923553]]),
 array([[8.71970506]]),
 array([[8.30853039]]),
 array([[8.23040794]]),
 array([[8.67373516]]),
 array([[8.71785551]]),
 array([[9.87634662]]),
 array([[10.48660786]]),
 array([[10.47569833]]),
 array([[10.86896344]]),
 array([[11.59038798]]),
 array([[11.18159473]]),
 array([[11.32174819]]),
 array([[11.00941129]]),
 array([[11.91674425]]),
 array([[12.41640679]]),
 array([[12.37774291]]),
 array([[12.25666139]]),
 array([[12.22062742]]),
 array([[11.32343992]]),
 array([[10.6569089]]),
 array([[10.09764726]]),
 array([[9.19151538]]),
 array([[8.30856399]]),
 array(

In [ ]:
class hmc():
    def __init__(data, lnp_grad, logprob, initial, mass_matrix=None, logpri=None):
        """
        get inputs and set up basic parameters
        """
        self.data = data
        self.initial = initial
        self.ndim = len(data[0])
        self.k = len(initial)
        self.get_mass_matrix(mass_matrix)
        self.lnp_grad = lnp_grad
        self.logprob = logprob
        
    def get_mass_matrix(self, mass_matrix=None):
        """
        get the inverse of the mass matrix
        """
        if mass_matrix is None:
            self.mass_matrix = np.identity(self.k)
            self.inverse_mass_matrix = np.identity(self.k)
        else:
            if len(mass_matrix) != self.k:
                print("Invalid mass matrix")
            elif len(mass_matrix) == 1:
                self.mass_matrix = mass_matrix
                self.inverse_mass_matrix = 1. / mass_matrix
            else:
                self.mass_matrix = mass_matrix
                self.inverse_mass_matrix = np.linalg.inv(mass_matrix)
            
    def grad_u(theta):
        """
        
        """
        df = self.data.copy()
        s = 0
        for x in df:
            s += self.lnp_grad(x, theta) 
        return -s / len(df)

    def u(theta):
        """
        
        """
        df = self.data.copy()
        s = 0 
        for x in df:
            s += self.logprob(x, theta)
        return -s / len(df)
    
    def kinetic_energy(self, r):
        """
        Get the kinetic energy given momentum
        """
        if self.ndim == 1:
            K = self.inverse_mass_matrix * r**2
        else:
            K = np.dot(r.T, np.dot(self.inverse_mass_matrix, r))
        return 0.5 * K
    
    def trajectory(initial,epsilon,length):
        """
        
        """
        r = np.random.multivariate_normal(np.zeros(self.k),self.mass_matrix,self.ndim).T
        theta0, r0 = initial, r 
        r0 = r - 0.5 * epsilon * grad_u(theta0)
        for i in range(length):
            theta_m = theta0 + epsilon * self.inverse_mass_matrix @ r0
            r_m = r0 - epsilon * grad_u(theta_m)
            theta0, r0 = theta_m, r_m
        r0 -= 0.5 * epsilon * grad_u(theta0)
        
        #M-H step
        mu = np.random.uniform(size=1)
        p = np.exp(u(theta0) + kinetic_energy(r0) - u(initial) - kinetic_energy(r))
    
        if mu < min(1,p):
            return theta0
        else:
            return initial

In [ ]:
r = np.random.multivariate_normal(np.zeros(5),np.identity(5),2).T
r

In [90]:
res = []
def grad_u(lnp_grad, data, theta):
    s = 0
    for x in data:
        s += lnp_grad(x, theta) 
    return -s / len(data)
def u(logprob, data, theta):
    s = 0 
    for x in data:
        s += logprob(x, theta)
    return -s / len(data)
def trajectory(initial, data, logprob, lnp_grad, epsilon,length):
    r = np.random.normal(0,1)
    theta0, r0 = initial.copy(), r.copy() 
    r0 = r - 0.5 * epsilon * grad_u(lnp_grad, data, theta0)
    for i in range(length):
        theta_m = theta0 + epsilon * r0
        r_m = r0 - epsilon * grad_u(lnp_grad, data, theta0)
        theta0, r0 = theta_m, r_m
    r0 -= 0.5*epsilon*grad_u(lnp_grad, data, theta0)
    #M-H step
    mu = np.random.uniform(size=1)
    p = np.exp(-u(logprob, data, theta0)-0.5 * r0 * r0 + u(logprob, data, initial) + 0.5 * r * r)
    if mu < min(1,p):
        return theta0
    else:
        return 0


In [91]:
res = []    
curr = [5]
for i in range(200):
    temp = trajectory(curr, data, logsig, sigma2grad, 0.1, 10)
    if temp != 0:
        curr = temp 
        res.append(temp[0])

AttributeError: 'float' object has no attribute 'copy'

In [128]:
import hmc
import autograd.numpy as np

ModuleNotFoundError: No module named 'hmc'